In [1]:
#Reading CSV file in this section
from PIL import Image
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import cv2
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.externals import joblib
from sklearn.metrics import f1_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from itertools import cycle
from sklearn.model_selection import RandomizedSearchCV

#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
#Reading the labels
%%time
d = Table.read('C:/Users/vip/Downloads/galaxy_zoo_labels.csv')
ID = np.asarray(d['GalaxyID'])
len(d)

Wall time: 786 ms


In [0]:
#Loading images from the database
%%time
imgdata1 = []    #list to store cropped data
#file = open('/content/gdrive/My Drive/galaxy_zoo/data_120_12k_centered.txt','wb') #To store and save in a file
for i in range(10000,11000):
        
    path='/content/images_training_rev1/'
    im = cv2.imread(path+str(ID[i])+'.jpg')

    '''
    NOTE: This part of the code is to identify the brightest point as the centre from cropping but we are using image center as the crop center at the moment.
    
    
    im = Image.open('/content/images_training_rev1/{0}.jpg'.format(ID[i]))    #reading individual images from ID(filenames)
    pix = im.load()
    pix_val = im.getdata()
    pix = np.asarray(pix_val)    #pixel data in array
    n,m = np.shape(pix_val)
    
    
    RGB_sum = np.zeros(179776)    #array of R+G+B for all pixels

    for i in range(179776):
        RGB_sum[i] = sum(pix[i,:])

    centre = np.argmax(RGB_sum)    #Finding brightest point in array
   
    for i in range(centre, n):    #loop to identify 
        if RGB_sum[i] <= 40:
            temp_w = i - centre
            break
        else:
            continue
    '''
    crop_img = im[151:271, 151:271]
    #cv2.imshow('Image', crop_img)
    #cv2.waitKey(0) 
    
    imgdata1.append((crop_img)/255)
    if i % 2000 == 0:
        print(i)
    
#pickle.dump((imgdata1), file)
#file.close()

10000
CPU times: user 1.97 s, sys: 300 ms, total: 2.27 s
Wall time: 2.76 s


In [3]:
## Opening the saved data file
#path='C:/Users/vip/Downloads/galaxy_zoo_images/images_training_rev1/'
f1 = open('C:/Users/vip/Downloads/data_120_centered.txt','rb')
data1 = pickle.load(f1)
f1.close()

In [4]:
#Reshaping the data in a format that can be used for training
X = np.asarray(data1).reshape((len(data1),43200))

In [5]:
## Reading labels for Q1 of classification tree categories
n = len(X)
q1 = np.zeros(3*n).reshape(n,3)
for i in range(n):
    q1[i] = [d['Class1.1'][i],d['Class1.2'][i],d['Class1.3'][i]]
    
maxi = []
for i in range(n):
    maxi.append(np.argmax(q1[i,:]))

y = np.zeros(3*n).reshape(n,3)
for i in range(n):
    y[i,maxi[i]] = 1

In [0]:
## Reading labels for Q6 of classification tree categories
n = len(X)
y = np.zeros(2*n).reshape(n,2)
for i in range(n):
    if d['Class6.1'][i]>=0.5:
        y[i,0] = 1
        #y[1] = 0
        
    else:
        y[i,1] = 1

In [7]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
## Load an already saved model here
loaded_model = joblib.load('C:/Users/vip/Downloads/RFbest_bi_model.sav')
result = loaded_model.score(X_test, y_test)
print(result)

0.8115


In [ ]:
#learning curve
#############
train_sizes = [i/10 for i in range(1,11,3)]

cv = ShuffleSplit(n_splits=5, test_size=0.15, random_state=0)

train_sizes, train_scores, validation_scores = learning_curve(
                                                   estimator = loaded_model, X=X,
                                                    y=y, train_sizes = train_sizes, cv = cv,)

In [0]:
#cnn.save('cnn_test.h5')
#load cnn model
cnn.save('Lenet_8.5k_20Epocs_SGD(0.01)_cnn_test.h5')

In [0]:
####################
#Plot ROC
####################
from sklearn.metrics import roc_curve, auc


# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
n_classes = 3
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i].ravel(), y_pred_test[:, i].ravel())
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred_test.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i+1, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

ValueError: ignored